In [32]:
from datetime import date, timedelta

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import sys

import math
import sklearn.metrics as skl_metrics

from datetime import timedelta
from logging import StreamHandler, DEBUG, Formatter, FileHandler, getLogger

logger = getLogger(__name__)
pd.options.mode.chained_assignment = None  # default='warn'

DIR = '../logs/'

log_fmt = Formatter('%(asctime)s %(name)s %(lineno)d [%(levelname)s][%(funcName)s] %(message)s ')
handler = StreamHandler()
handler.setLevel('INFO')
handler.setFormatter(log_fmt)
logger.addHandler(handler)

handler = FileHandler(DIR + 'train.py.log', 'a')
handler.setLevel(DEBUG)
handler.setFormatter(log_fmt)
logger.setLevel(DEBUG)
logger.addHandler(handler)

logger.info('start')

#------------------------------------------------------------------------------------#

df_train = pd.read_csv(
    '../input/train_1s.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
    float(u)) if float(u) > 0 else 0},
    parse_dates=["date"]
)

df_test = pd.read_csv(
    "../input/test_1s.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)



items = pd.read_csv(
    "../input/items.csv",
).set_index("item_nbr")




2018-01-01 13:53:51,127 __main__ 32 [INFO][<module>] start 
2018-01-01 13:53:51,127 __main__ 32 [INFO][<module>] start 
2018-01-01 13:53:51,127 __main__ 32 [INFO][<module>] start 
/home/zyp/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
df_train.shape

(2562153, 5)

In [132]:
df_2017 = df_train.loc[df_train.date>=pd.datetime(2017,5,1)]
#del df_train

promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
#del promo_2017_test, promo_2017_train

df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

items = items.reindex(df_2017.index.get_level_values(1))

In [175]:
df_2017_nbr = pd.DataFrame(df_2017["2017-06-01"].copy())
df_2017_nbr.reset_index(inplace = True)

In [176]:
df_2017.index.head(2)

AttributeError: 'MultiIndex' object has no attribute 'head'

In [177]:
df_2017.head(2)

date                2017-05-01  2017-05-02  2017-05-03  2017-05-04  \
store_nbr item_nbr                                                   
1         96995            0.0         0.0         0.0         0.0   
          99197            0.0         0.0         0.0         0.0   

date                2017-05-05  2017-05-06  2017-05-07  2017-05-08  \
store_nbr item_nbr                                                   
1         96995            0.0         0.0         0.0    0.693147   
          99197            0.0         0.0         0.0    0.000000   

date                2017-05-09  2017-05-10     ...      2017-08-06  \
store_nbr item_nbr                             ...                   
1         96995       0.693147    0.000000     ...        1.098612   
          99197       0.000000    0.693147     ...        0.000000   

date                2017-08-07  2017-08-08  2017-08-09  2017-08-10  \
store_nbr item_nbr                                                   
1         96995       1.098612         0.0    0.000000    0.693147   
          99197       1.098612         0.0    1.098612    0.000000   

date                2017-08-11  2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr item_nbr                                                              
1         96995            0.0         0.0         0.0         0.0         0.0  
          99197            0.0         0.0         0.0         0.0         0.0  

[2 rows x 107 columns]

In [178]:
promo_2017.head(2)

date                2017-05-01  2017-05-02  2017-05-03  2017-05-04  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   

date                2017-05-05  2017-05-06  2017-05-07  2017-05-08  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   

date                2017-05-09  2017-05-10     ...      2017-08-22  \
store_nbr item_nbr                             ...                   
1         96995          False       False     ...           False   
          99197          False       False     ...           False   

date                2017-08-23  2017-08-24  2017-08-25  2017-08-26  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   

date                2017-08-27  2017-08-28  2017-08-29  2017-08-30  2017-08-31  
store_nbr item_nbr                                                              
1         96995          False       False       False       False       False  
          99197          False       False       False       False       False  

[2 rows x 123 columns]

In [317]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]


def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
#       "item_nbr": df_2017_nbr.item_nbr,
#       "store_nbr": df_2017_nbr.store_nbr,
#       "date": (t2017 + delta), 
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values
    })
    
    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values

    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
    
#    X['dow'] = X['date'].dt.dayofweek
    if is_train:
        y = df_2017[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X


In [321]:

print("Preparing dataset...")
t2017 = date(2017, 6, 21)
X_l, y_l = [], []
for i in range(4):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l
X_val, y_val = prepare_dataset(date(2017, 7, 26))
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)


Preparing dataset...


In [257]:
#X_train['date'] =  pd.to_datetime(X_train['date'])
#X_train['dow'] = X_train['date'].dt.dayofweek

In [171]:
#del X_tmp["date"], X_tmp["item_nbr"], X_tmp["store_nbr"]

In [322]:
X_train.head(2)

,mean_14_2017,mean_3_2017,mean_7_2017,promo_14_2017,mean_4_dow0_2017,mean_4_dow1_2017,mean_4_dow2_2017,mean_4_dow3_2017,mean_4_dow4_2017,mean_4_dow5_2017,...,promo_6,promo_7,promo_8,promo_9,promo_10,promo_11,promo_12,promo_13,promo_14,promo_15
0,0.099021,0.00000,0.099021,0,0.346574,0.173287,0.173287,0.173287,0.173287,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,0.835944,1.24589,0.987960,0,0.794513,1.141087,0.447940,1.062124,0.549306,0.997246,...,0,0,0,0,0,0,0,0,0,0


In [323]:
features = X_train.columns.tolist()

In [344]:
features_all = X_train.columns.tolist()
for i in range(16):
    features_t = features_all.copy()
    for j in range(16):
        if j != i:
            features_t.remove("promo_{}".format(j))
    print(X_train[features_t].head(1))
#X_train.groupby(["dow"]).size()
#X_train["mean_4_dow{}_2017".format(0)].head(2)

   mean_14_2017  mean_3_2017  mean_7_2017  promo_14_2017  mean_4_dow0_2017  \
0      0.099021          0.0     0.099021              0          0.346574   

   mean_4_dow1_2017  mean_4_dow2_2017  mean_4_dow3_2017  mean_4_dow4_2017  \
0          0.173287          0.173287          0.173287          0.173287   

   mean_4_dow5_2017  mean_4_dow6_2017  promo_0  
0               0.0               0.0        0  
   mean_14_2017  mean_3_2017  mean_7_2017  promo_14_2017  mean_4_dow0_2017  \
0      0.099021          0.0     0.099021              0          0.346574   

   mean_4_dow1_2017  mean_4_dow2_2017  mean_4_dow3_2017  mean_4_dow4_2017  \
0          0.173287          0.173287          0.173287          0.173287   

   mean_4_dow5_2017  mean_4_dow6_2017  promo_1  
0               0.0               0.0        0  
   mean_14_2017  mean_3_2017  mean_7_2017  promo_14_2017  mean_4_dow0_2017  \
0      0.099021          0.0     0.099021              0          0.346574   

   mean_4_dow1_2017  me

In [334]:
y_train

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.69314718,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.38629436],
       [ 0.69314718,  1.38629436,  1.38629436, ...,  0.        ,
         0.69314718,  1.94591015],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.69314718,
         1.79175947,  1.60943791],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [338]:
logger.info('Training and predicting models...')

params = {
    'num_leaves': 31,
    'objective': 'regression',
    'min_data_in_leaf': 300,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 2,
    'metric': 'l2',
    'num_threads': 4
}

MAX_ROUNDS = 500
val_pred = []
test_pred = []
cate_vars = []

i = 0

print("=" * 50)
print("Step %d" % (i+1))
print("=" * 50)

dtrain = lgb.Dataset(
    X_train, label=y_train[:, i],
    categorical_feature=cate_vars
)
dval = lgb.Dataset(
     X_val, label=y_val[:, i], reference=dtrain,
     weight=items["perishable"] * 0.25 + 1,
     categorical_feature=cate_vars)



2018-01-01 20:21:38,530 __main__ 1 [INFO][<module>] Training and predicting models... 
2018-01-01 20:21:38,530 __main__ 1 [INFO][<module>] Training and predicting models... 
2018-01-01 20:21:38,530 __main__ 1 [INFO][<module>] Training and predicting models... 


Step 1


In [341]:
y_train

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.69314718,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.38629436],
       [ 0.69314718,  1.38629436,  1.38629436, ...,  0.        ,
         0.69314718,  1.94591015],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.69314718,
         1.79175947,  1.60943791],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [339]:

bst = lgb.train(
    params, dtrain, num_boost_round=MAX_ROUNDS,
    valid_sets=[dtrain, dval], early_stopping_rounds=50,
    verbose_eval=100
)



/home/zyp/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.


TypeError: 'NoneType' object is not iterable

In [ ]:
features_all = X_train.columns.tolist()
i = 0
features_t = features_all.copy()
for j in range(16):
    if j != i:
        features_t.remove("promo_{}".format(j))
X_train[features_t].head(1)

,mean_14_2017,mean_3_2017,mean_7_2017,promo_14_2017,promo_0,promo_1,promo_2,promo_3,promo_4,promo_5,promo_6,promo_7,promo_8,promo_9,promo_10,promo_11,promo_12,promo_13,promo_14,promo_15
0,1.601666,1.595831,1.326757,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
